<h4> This code is part of FLAUTO. It implements FedYogi. Date: 01/09/2025 </h4>
<h4> Contact: rakibul.haque@utsa.edu </h4>  
<h4> Cite as: R. U. Haque and P. Markopoulos,"Federated Learning with Automated Dual-Level Hyperparameter Tuning", 2025 <h4>

<H1>Import Libraries</H1>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torchvision.datasets import CIFAR10
from collections import Counter
import random
import copy
import torch.nn.functional as F
import os
import sys
import time
import pickle
import pandas as pd
from torch import Tensor
from typing import Type
import torchvision.models as models
from torchvision import transforms
from copy import deepcopy
from sklearn.metrics import confusion_matrix
import torchvision.models as models
from collections import Counter
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# Set font family for plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

<H1>Load Dataset</H1>

In [ ]:
subset_dataset = torch.load('val_dataset.pth',weights_only=False)
remaining_dataset = torch.load('test_dataset.pth',weights_only=False)

# Create DataLoaders
val_loader = DataLoader(subset_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(remaining_dataset, batch_size=32, shuffle=False)

<H1>IID</H1>

In [ ]:
def distribute_dataset_equally(dataset, num_clients):
    # Group data by class
    class_data = {}
    for data, label in dataset:
        if label not in class_data:
            class_data[label] = []
        class_data[label].append((data, label))

    # Distribute data
    client_data = [[] for _ in range(num_clients)]
    for label, data in class_data.items():
        data_len = len(data)
        base_size = data_len // num_clients
        remain = data_len - base_size * num_clients

        current_idx = 0
        for i in range(num_clients):
            end_idx = current_idx + base_size + (1 if i < remain else 0)
            client_data[i].extend(data[current_idx:end_idx])
            current_idx = end_idx


    print_iid_distribution(client_data)
    plot_iid_dataset(client_data,num_clients)

    return client_data

In [ ]:
#print client iid_data distribution
def print_iid_distribution(iid_datasets):
    # Check if the distribution is correct
    for i, client_data in enumerate(iid_datasets):
        print(f"Client {i + 1} data size: {len(client_data)}")
        class_counts = {j: 0 for j in range(10)}
        for _, label in client_data:
            class_counts[label] += 1
        print(f"Class distribution: {class_counts}")

In [ ]:
# Plot client iid_data distribution
def plot_iid_dataset(iid_datasets, num_clients):
    # CIFAR-10 class names
    class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
    # Calculate fraction of distribution for each class across clients
    fractions = []
    for client_data in iid_datasets:
        class_counts = {j: 0 for j in range(10)}
        for _, label in client_data:
            class_counts[label] += 1
        total_data = len(client_data)
        fractions.append([class_counts[i] / total_data for i in range(10)])
    fractions = np.array(fractions)
    # Define colors for each class
    colors = ["r", "g", "b", "c", "m", "y", "#FFA500", "#FF00FF", "#808080", "#00FF00"]
    # Generate the 3D bar chart
    fig = plt.figure(figsize=(10, 7))  # Adjust the size here
    ax = fig.add_subplot(111, projection='3d')

    xpos, ypos = np.meshgrid(np.arange(num_clients), np.arange(10), indexing="ij")
    for i in range(10):
        ax.bar3d(xpos[:, i], ypos[:, i], np.zeros_like(xpos[:, i]),0.75, 0.75, fractions[:, i],shade=True, color=colors[i])

    ax.set_xlabel('Client Number',labelpad=5)
    ax.set_ylabel('CIFAR-10 Classes',labelpad=15)
    ax.set_zlabel('Fraction of Distribution',labelpad=5)
    ax.set_xticks(np.arange(0.5, num_clients))
    ax.set_xticklabels([str(i+1) for i in range(num_clients)], rotation=45)
    ax.set_yticks(np.arange(0.5, 10))
    ax.set_yticklabels(class_names, rotation=-60)

    #ax.view_init(elev=40, azim=60)
    ax.view_init(elev=40, azim=10)

    plt.subplots_adjust(left=0.01, right=0.99, bottom=0.01, top=0.99)

    plt.show()

<H1>non-IID</H1>

In [ ]:
#Print the distribution non IID
def print_distribution(client_data):
    # Check distribution
    for l, client_data_value in enumerate(client_data):
        print(f"Client {l + 1} data size: {len(client_data_value)}")
        class_counts = {j: 0 for j in range(10)}
        for _, label in client_data_value:
            class_counts[label] += 1
        print(f"Class distribution: {class_counts}")

In [ ]:
# #Non IID code
def distribute_dataset_dirichlet(dataset, num_clients, alpha):
    # Group data by class
    class_data = {}
    for data, label in dataset:
        if label not in class_data:
            class_data[label] = []
        class_data[label].append((data, label))

    client_data = [[] for _ in range(num_clients)]
    for _, data_list in class_data.items():
        # Shuffle data for randomness
        np.random.shuffle(data_list)

        # Get proportions for data split based on Dirichlet distribution
        proportions = np.random.dirichlet([alpha]*num_clients)
        # print("Proportions: ", proportions)
        total_data = len(data_list)
        # print("total_data: ", total_data)
        data_splits = [int(proportions[i]*total_data) for i in range(num_clients)]
        # print("Data_Split: ", data_splits)

        # Adjust the splits to account for rounding errors
        # print("Before data_split:",data_splits)
        data_splits[-1] += total_data - sum(data_splits)
        print("After data_split:",data_splits)

        start_idx = 0
        for i, split in enumerate(data_splits):
            end_idx = start_idx + split
            client_data[i].extend(data_list[start_idx:end_idx])
            start_idx = end_idx
    print("Client Number: ",num_clients, "Alpha: ", alpha)
    print_distribution(client_data)
    plot_distribution(client_data, num_clients )
    return client_data

In [ ]:
#show the graphs for nonIID
def plot_distribution(client_datasets, num_clients):
    # CIFAR-10 class names
    class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
    # Calculate fraction of distribution for each class across clients
    fractions = []
    for client_data in client_datasets:
        class_counts = {j: 0 for j in range(10)}
        for _, label in client_data:
            class_counts[label] += 1
        total_data = len(client_data)
        # print(total_data)
        fractions.append([class_counts[i] / total_data for i in range(10)])
    fractions = np.array(fractions)
    # Define distinct colors for classes using a colormap
    colors = ["r", "g", "b", "c", "m", "y", "#FFA500", "#FF00FF", "#808080", "#00FF00"]
    # Generate the 3D bar chart
    fig = plt.figure(figsize=(10, 7))  # Adjust the size here
    ax = fig.add_subplot(111, projection='3d')

    xpos, ypos = np.meshgrid(np.arange(num_clients), np.arange(10), indexing="ij")

    for i in range(10):
        ax.bar3d(xpos[:, i], ypos[:, i], np.zeros_like(xpos[:, i]),0.75, 0.75, fractions[:, i],shade=True, color=colors[i])

    ax.set_xlabel('Client Number')
    ax.set_ylabel('CIFAR-10 Classes')
    ax.set_zlabel('Fraction of Distribution')
    ax.set_title('Distribution of CIFAR-10 Classes across Clients based on Dirichlet Distribution')
    ax.set_xticks(np.arange(0.5, num_clients))
    ax.set_xticklabels([str(i+1) for i in range(num_clients)], fontsize=12)
    ax.set_yticks(np.arange(0.5, 10))
    ax.set_yticklabels(class_names)
    plt.tight_layout()
    plt.show()

# Measures

In [ ]:
def accuracy(outp, target):
    """Computes accuracy"""
    with torch.no_grad():
        pred = torch.argmax(outp, dim=1)
        correct = pred.eq(target).float().sum().item()
        return 100.0 * correct / target.size(0)

In [ ]:
def Print(string, dictionary):
    first_key = next(iter(dictionary))
    first_value = dictionary[first_key]
    print(f"{string}:{first_key}: {first_value[0][0]}\n")

In [ ]:
def forbinus_norm_function(w_i):
    value = 0
    for k in w_i.keys():
        value += torch.linalg.norm(w_i[k])
    return value.item()

In [ ]:
def model_deviation_function(w_i, w_f):
    model_deviation = 0
    for k in w_i.keys():
        model_deviation += torch.linalg.norm(w_f[k].to(torch.float) - w_i[k].to(torch.float)) / torch.linalg.norm(w_i[k].to(torch.float))
    #print(model_deviation.item())
    return model_deviation.item()

# Model

In [ ]:
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.5)
        
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.relu5 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)  # Change output to 10 for CIFAR-10 classes

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.pool1(x)
        x = self.dropout1(x)
        x = self.relu3(self.conv3(x))
        x = self.relu4(self.conv4(x))
        x = self.pool2(x)
        x = self.dropout2(x)
        x = x.view(x.size(0), -1)
        x = self.relu5(self.fc1(x))
        x = self.dropout3(x)
        x = self.fc2(x)
        return x

# Train

In [ ]:
def train(i_weights, epochs, train_loader, le_rate, cli,roun, epoch_flag):
    global opti
    
    local_model = model().to(device)
    criterion = nn.CrossEntropyLoss()
    
    if opti=="adam":
        optimizer = torch.optim.Adam(local_model.parameters(), lr=le_rate)
    elif opti=="sgd":
        optimizer = torch.optim.SGD(local_model.parameters(), lr=le_rate)
    
    epoch_train_accuracy=0 
    epoch_train_loss=0
    epoch_test_accuracy=0
    epoch_test_loss=0
    epoch_rmd=0

    local_model.load_state_dict(i_weights)

    local_model.train()  # Set the model to training mode

    # initial weights cathing and printing
    initial_weights = {k: v.clone() for k, v in local_model.state_dict().items()}
    #Print("Model's inside the function Initial weights for client",initial_weights)

    # Training loop
    for epoch in range(epochs):
        epoch_flag=epoch_flag+1
        # gradients_this_epoch = {}
        total_samples = 0
        total_loss=0
        correct_samples = 0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            inputs, labels = inputs.to(device), labels.to(device)
            # Zero the parameter gradients
            optimizer.zero_grad()
            # Forward + backward + optimize
            outputs = local_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(local_model.parameters(), max_norm=1.0)
            
            total_loss += loss.item()
            optimizer.step()

            _, predicted = outputs.max(1)  # Get the index of the maximum value in outputs (predicted class)
            total_samples += labels.size(0)
            correct_samples += predicted.eq(labels).sum().item()
            
            #print(loss)
            if torch.isnan(loss):
                raise ValueError("Training stopped: NaN detected in loss.")
        
        if(total_samples!=0 and len(train_loader)!=0):
            epoch_accuracy = 100 * correct_samples / total_samples
            epoch_loss = total_loss / len(train_loader)
            #print("=========",epoch_loss, len(train_loader) )
        else:
            epoch_accuracy = 100 * correct_samples / (total_samples+1)
            epoch_loss = total_loss / (len(train_loader)+1)
            #print("=========",epoch_loss, len(train_loader)+1 )
        print(f"Round {roun}, cleint {cli+1}, epoch {epoch+1}: epoch_accuracy {epoch_accuracy}, epoch_loss {epoch_loss} ")
    
    f_weights = {k: v.clone() for k, v in local_model.state_dict().items()}

    #print(f"\n Round {roun}, cleint {cli}: epoch_accuracy {epoch_accuracy}, epoch_loss {epoch_loss} \n")
    epoch_train_accuracy=epoch_accuracy
    epoch_train_loss=epoch_loss
    epoch_test_accuracy, epoch_test_loss= test(f_weights, test_loader)
    
    
    epoch_rmd=model_deviation_function(initial_weights,f_weights)
    
    #saving data into dataframe
    epoch_data = [epoch_train_accuracy, epoch_train_loss, epoch_test_accuracy, epoch_test_loss, epoch_rmd]
    epoch_results.loc[len(epoch_results)] = epoch_data
    
    model_update = {}
    for key in local_model.state_dict():
        model_update[key] = torch.sub(i_weights[key], f_weights[key])
    
    return epoch_accuracy,epoch_loss, epoch_flag, model_update

# Test

In [ ]:
def test(w,data):
    lmodel = model().to(device)
    criterion = nn.CrossEntropyLoss()  # Assuming a classification task
    #optimizer = torch.optim.SGD(lmodel.parameters(), lr=learning_rate)
    lmodel.load_state_dict(w)
    lmodel.eval()

    #checking the weights
    tw = lmodel.state_dict()
    #Print("Model's before testing the weights in global model",tw)

    # Evaluation phase for test set
    acc_list = []
    loss_list = []

    with torch.no_grad():
        for j, data in enumerate(data, 0):
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            out = lmodel(images)
            # Calculate loss
            loss = criterion(out, labels)
            loss_list.append(loss.item())
            #calculate accuracy
            acc = accuracy(out, labels)
            acc_list.append(acc)
    test_loss = np.mean(loss_list)
    test_accuracy = np.mean(acc_list)
    #print("Model's Test accuracy : {:.2f}%".format(test_accuracy))
    return test_accuracy, test_loss

# FL Structure

In [ ]:
def average_updates(w, n_k):
    w_avg = deepcopy(w[0])
    for key in w_avg.keys():
        w_avg[key] = torch.mul(w_avg[key], n_k[0])
        for i in range(1, len(w)):
            w_avg[key] = torch.add(w_avg[key], w[i][key], alpha=n_k[i])
        w_avg[key] = torch.div(w_avg[key], sum(n_k))
    return w_avg

In [ ]:
def federated_learning(i_w, data_client, C, P, R, E, learning_rate, b_size, beta_1=0.9, beta_2=0.999, tau=1e-3, epsilon=1e-8):
    
    global total_clients_list, participating_client_list, m, v, t
    r_flag=0
    global_model.load_state_dict(i_w)
    t = 0  # Time step for FedYogi

    # loop for round
    for r in range(1, R+1):
        round_train_accuracy = 0
        round_train_loss = 0
        round_test_accuracy = 0
        round_test_loss = 0
        epoch_flag = 0

        # Saving initial weights for spiking model
        i_w = {k: v.clone() for k, v in global_model.state_dict().items()}

        # Collecting weights and results
        data_size = []
        all_clients_updates = []
        train_accuracy_list = []
        train_loss_list = []
        
        # Randomly select clients
        selected_clients = random.sample(total_clients_list, P)
        participating_client_list.append(selected_clients)

        # Loop for client
        for c, data in enumerate(data_client):
            if c in selected_clients:
                train_loader = torch.utils.data.DataLoader(data, batch_size=b_size, shuffle=True, drop_last=False)
                
                # Train model
                train_accuracy, train_loss, epoch_flag, model_update = train(i_w, E, train_loader, learning_rate, c, r, epoch_flag)

                train_accuracy_list.append(train_accuracy)
                train_loss_list.append(train_loss)
                
                all_clients_updates.append(model_update)
                data_size.append(len(train_loader))
            else:
                print(f"Client {c+1} is not selected")
        
        round_epoch = epoch_flag
        round_train_loss = sum(train_loss_list) / len(train_loss_list)
        round_train_accuracy = sum(train_accuracy_list) / len(train_accuracy_list)
        print(f"Model's Round: {r}, train accuracy of model: {round_train_accuracy}, train loss of model: {round_train_loss} \n\n")

        # Aggregate the updates using a weighted average
        update_avg = average_updates(all_clients_updates, data_size)

        # Initialize moment vectors if not done
        if m is None:
            m = {key: torch.zeros_like(param) for key, param in update_avg.items()}
            v = {key: torch.zeros_like(param) for key, param in update_avg.items()}

        # Yogi update rule for each parameter
        t += 1
        for key in i_w:
            # Update biased first moment estimate
            m[key] = beta_1 * m[key] + (1 - beta_1) * update_avg[key]
            
            # Update biased second raw moment estimate (Yogi adjustment)
            v[key] = v[key] - (1 - beta_2) * (update_avg[key] ** 2) * torch.sign(v[key] - update_avg[key] ** 2)
            
            # Compute bias-corrected first moment estimate
            m_hat = m[key] / (1 - beta_1 ** t)
            
            # Compute bias-corrected second moment estimate (with Yogi adjustment)
            v_hat = v[key] / (1 - beta_2 ** t)
            
            # Update weights using Yogi rule
            i_w[key] -= learning_rate * m_hat / (torch.sqrt(v_hat) + epsilon)

        # Test the model on global test set
        round_test_accuracy, round_test_loss = test(i_w, test_loader)
        print(f"Model's Round: {r}, test accuracy of model: {round_test_accuracy}, test loss of model: {round_test_loss} \n\n")
        
        round_val_accuracy, round_val_loss=test(i_w, val_loader)
        print(f"Model's Round: {r}, val accuracy of model: {round_val_accuracy}, val loss of model: {round_val_loss} \n\n")
        
        list_accuracy.append(round_train_accuracy)
        list_loss.append(round_train_loss)
        list_val_accuracy.append(round_val_accuracy)
        list_val_loss.append(round_val_loss)
        list_test_accuracy.append(round_test_accuracy)
        list_test_loss.append(round_test_loss)
        
        
        # Model deviation calculation
        round_rmd = model_deviation_function(i_w, global_model.state_dict())
        
        # Save data into dataframe
        round_data = [round_train_accuracy, round_train_loss, round_test_accuracy, round_test_loss, round_rmd, round_epoch, round_val_accuracy, round_val_loss]
        round_results.loc[len(round_results)] = round_data
        
        print((list_val_loss[r] > list_val_loss[r-1]) and (list_loss[r] > list_loss[r-1]))

        # Load the updated weights into the global model
        global_model.load_state_dict(i_w)
        print("Round", r, "completed")


# Main Function

# Define parameters

In [ ]:
#===========================Parameters==============================================================
client_no=20
participating_client=20
epochs=5
learning_rate=0.0001
round_no=30
batch_size=128
distributions = "non_iid" # 'non_iid'
data_class=10
alpha=0.5

opti="sgd"
# opti="adam"


v=None
m=None
t=0

method="fed_yogi"

# List of clients
total_clients_list = list(range(0, client_no))
# print(total_cleints_list)
participating_client_list=[]


list_accuracy=[]
list_loss=[]
list_test_accuracy=[]
list_test_loss=[]
list_val_accuracy=[]
list_val_loss=[]

# Define dataframe for round results
round_columns = ['train_accuracy', 'train_loss', 'test_accuracy', 'test_loss', 'rmd', 'epoch', 'val_accuracy', 'val_loss']
round_results = pd.DataFrame(columns=round_columns)

# Define dataframe for epoch results
epoch_columns = ['train_accuracy', 'train_loss', 'test_accuracy', 'test_loss', 'rmd']
epoch_results = pd.DataFrame(columns=epoch_columns)

#===================================loading the saved weight list====================================================
global_model = model().to(device)
# initial_weights={k: v.clone() for k, v in global_model.state_dict().items()}

# Save the initial weights
file_path = "s_cnn.pth"
# torch.save(initial_weights, file_path)
initial_weights=torch.load(file_path,weights_only=True)
Print("Model's initial weights", initial_weights)


<H1>Divide data among cleints</H1>

In [ ]:
# Load client_datasets from a file
if distributions == 'iid':
    with open('20_client_datasets_IID.pkl', 'rb') as f:
        client_datasets = pickle.load(f)

elif distributions == 'non_iid' and alpha==0.5:
    with open('20_client_datasets_non_IID_0_5.pkl', 'rb') as f:
        client_datasets = pickle.load(f)
        
print("client_datasets loaded successfully.")
print_distribution(client_datasets)

<H1>Round zero</H1>

In [ ]:
#train accuracy for cleints
round_train_accuracy=0
round_train_loss=0
train_accuracy_list=[]
train_loss_list=[]
for c, data in enumerate(client_datasets):
    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)
    train_accuracy, train_loss=test(initial_weights, train_loader)
    train_accuracy_list.append(train_accuracy)
    train_loss_list.append(train_loss)
round_train_accuracy=(sum(train_accuracy_list)/len(train_accuracy_list))
round_train_loss=(sum(train_loss_list)/len(train_loss_list))


list_accuracy.append(round_train_accuracy)
list_loss.append(round_train_loss)

#test accuracy for server
round_test_accuracy=0
round_test_loss=0
test_accuracy,test_loss=test(initial_weights,test_loader)
val_accuracy,val_loss=test(initial_weights,val_loader)

print(f"{test_accuracy}, {test_loss}, {val_accuracy}, {val_loss}  ")
round_test_accuracy=(test_accuracy)
round_test_loss=(test_loss)

round_rmd=0
round_epoch=0

round_data = [round_train_accuracy, round_train_loss, round_test_accuracy, round_test_loss, round_rmd, round_epoch, val_accuracy,val_loss]
round_results.loc[len(round_results)] = round_data

list_val_accuracy.append(val_accuracy)
list_val_loss.append(val_loss)
list_test_accuracy.append(test_accuracy)
list_test_loss.append(test_loss)


Print("initial_weights", initial_weights)
print(f' train accuracy: {round_train_accuracy}\n train_loss: {round_train_loss}\n test_accuracy: {round_test_accuracy}\n test_loss: {round_test_loss}')

<H1>Run FL</H1>

In [1]:
federated_learning(initial_weights, client_datasets, client_no, participating_client, round_no, epochs, learning_rate, batch_size)

Round 9, cleint 16, epoch 10: epoch_accuracy 25.81686163775716, epoch_loss 2.278198254414094 
Round 9, cleint 17, epoch 1: epoch_accuracy 7.92712949286066, epoch_loss 2.3107183054089546 
Round 9, cleint 17, epoch 2: epoch_accuracy 8.911866075824717, epoch_loss 2.309665411710739 
Round 9, cleint 17, epoch 3: epoch_accuracy 9.10881339241753, epoch_loss 2.3096448332071304 
Round 9, cleint 17, epoch 4: epoch_accuracy 8.911866075824717, epoch_loss 2.3096076622605324 
Round 9, cleint 17, epoch 5: epoch_accuracy 8.665681930083702, epoch_loss 2.309071071445942 
Round 9, cleint 17, epoch 6: epoch_accuracy 8.911866075824717, epoch_loss 2.3088759556412697 
Round 9, cleint 17, epoch 7: epoch_accuracy 9.404234367306746, epoch_loss 2.308785617351532 
Round 9, cleint 17, epoch 8: epoch_accuracy 9.650418513047759, epoch_loss 2.308116726577282 
Round 9, cleint 17, epoch 9: epoch_accuracy 9.256523879862137, epoch_loss 2.3086569011211395 
Round 9, cleint 17, epoch 10: epoch_accuracy 8.911866075824717, ep

Round 10, cleint 5, epoch 5: epoch_accuracy 10.56422569027611, epoch_loss 2.2839923804661013 
Round 10, cleint 5, epoch 6: epoch_accuracy 10.384153661464586, epoch_loss 2.2844385875845856 
Round 10, cleint 5, epoch 7: epoch_accuracy 11.254501800720288, epoch_loss 2.2828446244293787 
Round 10, cleint 5, epoch 8: epoch_accuracy 12.064825930372148, epoch_loss 2.283466415585212 
Round 10, cleint 5, epoch 9: epoch_accuracy 12.244897959183673, epoch_loss 2.282779689105052 
Round 10, cleint 5, epoch 10: epoch_accuracy 11.914765906362545, epoch_loss 2.282703804519941 
Round 10, cleint 6, epoch 1: epoch_accuracy 16.011971567527123, epoch_loss 2.3085516010011946 
Round 10, cleint 6, epoch 2: epoch_accuracy 16.011971567527123, epoch_loss 2.308247685432434 
Round 10, cleint 6, epoch 3: epoch_accuracy 16.610549943883278, epoch_loss 2.3082288276581537 
Round 10, cleint 6, epoch 4: epoch_accuracy 17.171717171717173, epoch_loss 2.307196577390035 
Round 10, cleint 6, epoch 5: epoch_accuracy 15.11410400

Round 10, cleint 14, epoch 2: epoch_accuracy 10.790144435004247, epoch_loss 2.29400406012664 
Round 10, cleint 14, epoch 3: epoch_accuracy 9.664401019541206, epoch_loss 2.29408614699905 
Round 10, cleint 14, epoch 4: epoch_accuracy 9.749362786745964, epoch_loss 2.2937922896565617 
Round 10, cleint 14, epoch 5: epoch_accuracy 10.960067969413764, epoch_loss 2.293875066009728 
Round 10, cleint 14, epoch 6: epoch_accuracy 10.598980458793543, epoch_loss 2.2938735162889636 
Round 10, cleint 14, epoch 7: epoch_accuracy 11.363636363636363, epoch_loss 2.293366602949194 
Round 10, cleint 14, epoch 8: epoch_accuracy 10.875106202209006, epoch_loss 2.293363957791715 
Round 10, cleint 14, epoch 9: epoch_accuracy 11.342395921835173, epoch_loss 2.2930220977680102 
Round 10, cleint 14, epoch 10: epoch_accuracy 11.639762107051826, epoch_loss 2.292643534170615 
Round 10, cleint 15, epoch 1: epoch_accuracy 5.018489170628632, epoch_loss 2.3046792189280194 
Round 10, cleint 15, epoch 2: epoch_accuracy 5.916

Round 11, cleint 2, epoch 6: epoch_accuracy 9.964947421131697, epoch_loss 2.282915137708187 
Round 11, cleint 2, epoch 7: epoch_accuracy 9.26389584376565, epoch_loss 2.282168887555599 
Round 11, cleint 2, epoch 8: epoch_accuracy 10.41562343515273, epoch_loss 2.281947374343872 
Round 11, cleint 2, epoch 9: epoch_accuracy 9.914872308462694, epoch_loss 2.2822734490036964 
Round 11, cleint 2, epoch 10: epoch_accuracy 10.615923885828742, epoch_loss 2.2822336703538895 
Round 11, cleint 3, epoch 1: epoch_accuracy 2.5302220972729828, epoch_loss 2.3210509845188687 
Round 11, cleint 3, epoch 2: epoch_accuracy 2.3615406241214507, epoch_loss 2.321897872856685 
Round 11, cleint 3, epoch 3: epoch_accuracy 2.58644925499016, epoch_loss 2.3215835775647844 
Round 11, cleint 3, epoch 4: epoch_accuracy 2.6426764127073374, epoch_loss 2.320816401924406 
Round 11, cleint 3, epoch 5: epoch_accuracy 2.7551307281416926, epoch_loss 2.3211727482931956 
Round 11, cleint 3, epoch 6: epoch_accuracy 2.16474557211133,

Round 11, cleint 11, epoch 5: epoch_accuracy 5.873715124816447, epoch_loss 2.316647529602051 
Round 11, cleint 11, epoch 6: epoch_accuracy 5.726872246696035, epoch_loss 2.3166496753692627 
Round 11, cleint 11, epoch 7: epoch_accuracy 5.873715124816447, epoch_loss 2.315811352296309 
Round 11, cleint 11, epoch 8: epoch_accuracy 4.405286343612334, epoch_loss 2.3167821060527456 
Round 11, cleint 11, epoch 9: epoch_accuracy 6.901615271659325, epoch_loss 2.315928025679155 
Round 11, cleint 11, epoch 10: epoch_accuracy 6.167400881057269, epoch_loss 2.317548860203136 
Round 11, cleint 12, epoch 1: epoch_accuracy 4.4619422572178475, epoch_loss 2.3113217651844025 
Round 11, cleint 12, epoch 2: epoch_accuracy 3.3464566929133857, epoch_loss 2.312833249568939 
Round 11, cleint 12, epoch 3: epoch_accuracy 4.8556430446194225, epoch_loss 2.3127019107341766 
Round 11, cleint 12, epoch 4: epoch_accuracy 3.937007874015748, epoch_loss 2.3118416170279183 
Round 11, cleint 12, epoch 5: epoch_accuracy 3.4776

...

Round 99, cleint 3, epoch 1: epoch_accuracy 24.6274950801237, epoch_loss 2.197826930454799 
Round 99, cleint 3, epoch 2: epoch_accuracy 24.6274950801237, epoch_loss 2.1965074964932034 
Round 99, cleint 3, epoch 3: epoch_accuracy 25.330334551588418, epoch_loss 2.19468754529953 
Round 99, cleint 3, epoch 4: epoch_accuracy 24.65560865898229, epoch_loss 2.194921855415617 
Round 99, cleint 3, epoch 5: epoch_accuracy 24.40258644925499, epoch_loss 2.195716840880258 
Round 99, cleint 3, epoch 6: epoch_accuracy 24.768062974416644, epoch_loss 2.190473198890686 
Round 99, cleint 3, epoch 7: epoch_accuracy 25.189766657295475, epoch_loss 2.193991793053491 
Round 99, cleint 3, epoch 8: epoch_accuracy 23.840314872083216, epoch_loss 2.189416595867702 
Round 99, cleint 3, epoch 9: epoch_accuracy 24.515040764689346, epoch_loss 2.1909901925495694 
Round 99, cleint 3, epoch 10: epoch_accuracy 24.768062974416644, epoch_loss 2.192946727786745 
Round 99, cleint 4, epoch 1: epoch_accuracy 15.506653019447288, 

Round 99, cleint 11, epoch 9: epoch_accuracy 6.167400881057269, epoch_loss 2.3592923337763008 
Round 99, cleint 11, epoch 10: epoch_accuracy 5.580029368575624, epoch_loss 2.366126385602084 
Round 99, cleint 12, epoch 1: epoch_accuracy 2.8215223097112863, epoch_loss 2.318374365568161 
Round 99, cleint 12, epoch 2: epoch_accuracy 3.477690288713911, epoch_loss 2.318133463462194 
Round 99, cleint 12, epoch 3: epoch_accuracy 2.7559055118110236, epoch_loss 2.3219318985939026 
Round 99, cleint 12, epoch 4: epoch_accuracy 2.8871391076115485, epoch_loss 2.315067857503891 
Round 99, cleint 12, epoch 5: epoch_accuracy 3.477690288713911, epoch_loss 2.316908965508143 
Round 99, cleint 12, epoch 6: epoch_accuracy 3.0183727034120733, epoch_loss 2.3154345750808716 
Round 99, cleint 12, epoch 7: epoch_accuracy 3.8057742782152233, epoch_loss 2.3148696223894754 
Round 99, cleint 12, epoch 8: epoch_accuracy 2.7559055118110236, epoch_loss 2.3228600223859153 
Round 99, cleint 12, epoch 9: epoch_accuracy 3.1

Round 99, cleint 20, epoch 6: epoch_accuracy 24.289405684754524, epoch_loss 2.2197746839679655 
Round 99, cleint 20, epoch 7: epoch_accuracy 24.237726098191214, epoch_loss 2.218818531661737 
Round 99, cleint 20, epoch 8: epoch_accuracy 23.385012919896642, epoch_loss 2.2187206784232716 
Round 99, cleint 20, epoch 9: epoch_accuracy 24.031007751937985, epoch_loss 2.219574068413406 
Round 99, cleint 20, epoch 10: epoch_accuracy 24.62532299741602, epoch_loss 2.2150585690482716 
Model's Round: 99, train accuracy of model: 16.86304772159704, train loss of model: 2.2481104704677017 


Model's Round: 99, test accuracy of model: 20.460992907801423, test loss of model: 2.2129545363974064 


Model's Round: 99, val accuracy of model: 18.789556962025316, val loss of model: 2.2155042117155053 


False
0
Round 99 completed
Round 100, cleint 1, epoch 1: epoch_accuracy 25.8298755186722, epoch_loss 2.1683613254177954 
Round 100, cleint 1, epoch 2: epoch_accuracy 25.414937759336098, epoch_loss 2.165250832

Round 100, cleint 8, epoch 10: epoch_accuracy 15.64461612747465, epoch_loss 2.1638479232788086 
Round 100, cleint 9, epoch 1: epoch_accuracy 12.906178489702517, epoch_loss 2.2529702254704067 
Round 100, cleint 9, epoch 2: epoch_accuracy 12.99771167048055, epoch_loss 2.258002894265311 
Round 100, cleint 9, epoch 3: epoch_accuracy 13.135011441647597, epoch_loss 2.2522296292441233 
Round 100, cleint 9, epoch 4: epoch_accuracy 13.180778032036613, epoch_loss 2.2523455483572823 
Round 100, cleint 9, epoch 5: epoch_accuracy 13.31807780320366, epoch_loss 2.249290704727173 
Round 100, cleint 9, epoch 6: epoch_accuracy 13.68421052631579, epoch_loss 2.255009617124285 
Round 100, cleint 9, epoch 7: epoch_accuracy 12.540045766590389, epoch_loss 2.2520764282771517 
Round 100, cleint 9, epoch 8: epoch_accuracy 12.585812356979405, epoch_loss 2.2517921583993092 
Round 100, cleint 9, epoch 9: epoch_accuracy 12.860411899313501, epoch_loss 2.2503278800419397 
Round 100, cleint 9, epoch 10: epoch_accuracy 

Round 100, cleint 17, epoch 7: epoch_accuracy 7.877892663712457, epoch_loss 2.376095049083233 
Round 100, cleint 17, epoch 8: epoch_accuracy 7.336287543082226, epoch_loss 2.3764427974820137 
Round 100, cleint 17, epoch 9: epoch_accuracy 6.696208764155588, epoch_loss 2.377907909452915 
Round 100, cleint 17, epoch 10: epoch_accuracy 7.434761201378631, epoch_loss 2.379936844110489 
Round 100, cleint 18, epoch 1: epoch_accuracy 38.130200685266765, epoch_loss 2.0467944430808225 
Round 100, cleint 18, epoch 2: epoch_accuracy 37.99967368249307, epoch_loss 2.0447687258323035 
Round 100, cleint 18, epoch 3: epoch_accuracy 37.65704030021211, epoch_loss 2.043504452953736 
Round 100, cleint 18, epoch 4: epoch_accuracy 37.95072605645293, epoch_loss 2.0414741237958274 
Round 100, cleint 18, epoch 5: epoch_accuracy 37.46124979605156, epoch_loss 2.04180991028746 
Round 100, cleint 18, epoch 6: epoch_accuracy 37.673356175558816, epoch_loss 2.0396453688542047 
Round 100, cleint 18, epoch 7: epoch_accura

# Reuslts

In [ ]:
# Define the folder and file name
folder_name = f"{method}_{opti}_{learning_rate}_{participating_client}_{client_no}_{distributions}_{alpha}"  # Folder where the Excel file will be saved
file_name = "round_results.xlsx"


# Check if the folder exists, if not, create it
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Full path where the Excel file will be saved
file_path = os.path.join(folder_name, file_name)

round_results.to_excel(file_path, index=False)

print("DataFrame successfully written for round results.")

In [ ]:
# Define the folder and file name
folder_name =  f"{method}_{opti}_{learning_rate}_{participating_client}_{client_no}_{distributions}_{alpha}"   # Folder where the Excel file will be saved
file_name = "epoch_results.xlsx"

# Check if the folder exists, if not, create it
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Full path where the Excel file will be saved
file_path = os.path.join(folder_name, file_name)

epoch_results.to_excel(file_path, index=False)

print("DataFrame successfully written for epoch results.")